In [114]:
# To download top 50 stocks

In [144]:
# libraries 

from getData import get_data, get_ohlcv
from technical_indicator import LR, MACD, RSI, BB, ATR, ADX, RENKO, anomaly, historical_volatility, get_support_resistance_levels, VWAP
from Returns import cummulative_returns, CAGR, volatility, SHARPE, SORTINO, MAXDROWDOWN, CALMAR
from signals import stratgey1_mean_reversion


from plotly.subplots import make_subplots
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import copy
import os

%load_ext autoreload
%autoreload 2
pd.set_option('mode.chained_assignment', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Getting Data

In [145]:
# tickers = ['ICICIBANK.NS']
# tickers

# tickers = [
#     'ADANIPORTS.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS',
#     'BAJAJFINSV.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'CIPLA.NS', 'COALINDIA.NS',
#     'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GAIL.NS', 'GRASIM.NS',
#     'HCLTECH.NS', 'HDFCBANK.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDPETRO.NS',
#     'HINDUNILVR.NS', 'HDFC.NS', 'ITC.NS', 'ICICIBANK.NS', 'IBULHSGFIN.NS',
#     'IOC.NS', 'INDUSINDBK.NS', 'INFY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS',
#     'LT.NS', 'M&M.NS', 'MARUTI.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS',
#     'RELIANCE.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATAMOTORS.NS',
#     'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'UPL.NS', 'ULTRACEMCO.NS',
#     'VEDL.NS', 'WIPRO.NS', 'YESBANK.NS', 'ZEEL.NS'
# ]

tickers = [
    'ADANIPORTS.NS', 'ASIANPAINT.NS', 
    'CIPLA.NS', 
    'ITC.NS', 'ICICIBANK.NS',
    'IOC.NS',  'INFY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS',
    
    'RELIANCE.NS', 
    'VEDL.NS', 'WIPRO.NS'
]


In [146]:
# Store data in dictionary
ohlcv_dict = {}
for stock in tickers:
    ohlcv_dict[stock] = get_ohlcv(ticker=stock, period = '5y', interval='1d', multi_level_index=False)
    # ohlcv_dict[stock] = ohlcv_dict[stock].between_time('09:35', '16:00')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [147]:
data = copy.deepcopy(ohlcv_dict)

# Calculation on Data

In [148]:
for stock_name in tickers:
    cummulative_returns(data[stock_name])
    ATR(data[stock_name])
    anomaly(data[stock_name])
    support, resistnace  = get_support_resistance_levels(data[stock_name])
    data[stock_name]['support'] = support
    data[stock_name]['resistance'] = resistnace
    historical_volatility(data[stock_name])
    historical_volatility(data[stock_name])
    RSI(data[stock_name])
    VWAP(data[stock_name])
    BB(data[stock_name])
    data[stock_name]['Close'] = round(data[stock_name]['Close'],2)
    data[stock_name]['Signal'] = data[stock_name].apply(stratgey1_mean_reversion, axis=1)
    print(f'Now processing {[stock_name]}')

Now processing ['ADANIPORTS.NS']
Now processing ['ASIANPAINT.NS']
Now processing ['CIPLA.NS']
Now processing ['ITC.NS']
Now processing ['ICICIBANK.NS']
Now processing ['IOC.NS']
Now processing ['INFY.NS']
Now processing ['JSWSTEEL.NS']
Now processing ['KOTAKBANK.NS']
Now processing ['RELIANCE.NS']
Now processing ['VEDL.NS']
Now processing ['WIPRO.NS']


In [149]:
# data

In [152]:
for stock_name in tickers:
    df = data[stock_name]
    resistance = df.loc[df['resistance']>1]['resistance'].tail(5)
    resistance.name = 'Close'
    
    resistance_levels = [
        {"date": date.strftime("%Y-%m-%d"), "Close": round(Close,2)}
        for date, Close in resistance.items()
    ]
    # resistance_levels
    
    support = df.loc[df['support']>1]['support'].tail(5)
    support.name = 'Close'
    
    support_levels = [
        {"date": date.strftime("%Y-%m-%d"), "Close": round(Close,2)}
        for date, Close in support.items()
    ]
    
    
    # Buy
    buy = df.loc[df['Signal'] ==1]['Close'].tail(5)
    buy.name = 'Close'
    
    # Sell
    sell = df.loc[df['Signal'] ==-1]['Close'].tail(5)
    sell.name = 'Close'

    fig = go.Figure()
    #Close
    fig.add_trace(go.Scatter(
        x=data[stock_name].index,
        y=data[stock_name]['Close'],
        mode='lines',  # Set the mode to 'lines' for a line chart
        name='Close'# Use the name for each line from the dictionary
    ))   

    fig.add_trace(go.Scatter(
        x=data[stock_name].index,
        y=data[stock_name]['MB'],
        mode='lines',  # Set the mode to 'lines' for a line chart
        name='MB'# Use the name for each line from the dictionary
    ))
    
    fig.add_trace(go.Scatter(
        x=data[stock_name].index,
        y=data[stock_name]['UB'],
        mode='lines',  # Set the mode to 'lines' for a line chart
        name='UB'# Use the name for each line from the dictionary
    ))
        

    fig.add_trace(go.Scatter(
        x=data[stock_name].index,
        y=data[stock_name]['LB'],
        mode='lines',  # Set the mode to 'lines' for a line chart
        name='LB'# Use the name for each line from the dictionary
    )) 
    
    
    for level in support_levels:
        fig.add_hline(
            y=level["Close"],
            line=dict(color="green", dash="dash"),
            annotation_text=f"Support ({level['Close']})",
            annotation_position="bottom left"
        )
    
    # Add resistance lines
    for level in resistance_levels:
        fig.add_hline(
            y=level["Close"],
            line=dict(color="red", dash="dash"),
            annotation_text=f"Resistance ({level['Close']})",
                    annotation_position="top left"
        )
    
    #Buy
    fig.add_trace(go.Scatter(
        x=buy.index,
        y=buy.values,
        mode='markers',
        name='Buy Signal',
        marker=dict(color='green', size=20, symbol='triangle-up')
        ))
    
    
    #Sell
    fig.add_trace(go.Scatter(
        x=sell.index,
        y=sell.values,
        mode='markers',
        name='Sell Signal',
        marker=dict(color='red', size=20, symbol='triangle-down')
        ))
    
    # Customize the layout
    fig.update_layout(
        title=f"{stock_name} Signal Chart",
        xaxis_title="Year",
        yaxis_title="Cumulative Return",
        autosize=False,
        width=1100,
        height=800,
        template='plotly_white'
    )
        
    
    
    # Path where you want to save
    output_dir = "../PlotlyOutput"
    
    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Full file path
    file_path = os.path.join(output_dir, f"{stock_name}.png")
    
    # Save the figure
    fig.write_image(file_path)

    print(f"Output saved for {stock_name}")

Output saved for ADANIPORTS.NS
Output saved for ASIANPAINT.NS
Output saved for CIPLA.NS
Output saved for ITC.NS
Output saved for ICICIBANK.NS
Output saved for IOC.NS
Output saved for INFY.NS
Output saved for JSWSTEEL.NS
Output saved for KOTAKBANK.NS
Output saved for RELIANCE.NS
Output saved for VEDL.NS
Output saved for WIPRO.NS
